In [233]:
import ess # self-built module for battery related function
import preprocess # self-built module for preprocess related function
import rl_model # self-built module for reinforcement learning related function
import predictive_model # self-built module for prediction related function
import numpy as np
import pandas as pd
from scipy.optimize import linprog

import time
import influxdb
import warnings
import datetime
warnings.filterwarnings("ignore")

# Declare Function 

In [2]:
import numpy as np
from scipy.optimize import linprog

In [3]:
# Linear programming optimization

def optimize_linprog(pv, load, price, battery, num_timesteps):
    
    """
      1. x(t) - action at time t (within the range [-1, 1])
         p(t) - real time price at time t
         k - wear and tear cost of ESS
         P - per unit rated power of ESS
         E - per unit rated energy of ESS

         objective: minimize the transactive cost across the next 24 hours
                    min (x1*(p1+k) + x2*(p2+k) + ... + x24*(p24+k))
         inequality constraint: operation limit - 0 <= x(t) * P / E + SOC(t) <= 1
         bounds: upper and lower bound of action [-1, 1]
         return: action of next 24 hours
      2. the actual ESS output power = action * P
      3. the SOC of ESS is updated after each hour and the linear programming optimization 
         is performed every hour with the updated data - model prediction control
    """

    # coefficients of the linear objective function to be minimized
    # coefficients is normalized with load and pv such that 
    # ESS will react normally if there is no PV
    # ESS tend to store more PV when PV is much higher than load
    c = (price + battery.wear_cost) * load / (load + pv) * battery.P_rated

    # Inequality constraint - 0 <= x1 / E + SOC1 <= 1
    Aup = np.ones((24, 24)) * battery.P_rated / battery.E_rated 
    Aup = np.tril(Aup)
    Adown = np.flipud(Aup) * -1
    A = np.concatenate((Aup, Adown), axis=0) 
    bup = np.ones((24,)) - battery.current_SOC
    bdown = np.zeros((24,)) + battery.current_SOC - battery.target_SOC
    b = np.concatenate((bup, bdown), axis=0)

    # upper and lower bounds of the actions
    bounds = []
    for timestep in range(num_timesteps):
        bounds.append((-1, 1))

    # linear programming
    res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='revised simplex')
    actions = res.x
    
    return actions

In [275]:
# Compute the monetary cost or benefit with/ without ESS/ battery
# compute_benefit(x[:, 0:1], x[:, 1:2], x[:, 2:3], np.array([[0.]*len(x)]).transpose(), battery=None)

def compute_benefit(pv_energy, load_energy, price, battery_energy, battery=None):
    
    # Reshape from 1d to 2d array and inverse_transform
    if len(pv_energy.shape) == 1:
        load_energy = np.reshape(load_energy, (1, -1))
        pv_energy = np.reshape(pv_energy, (1, -1))
        battery_energy = np.reshape(battery_energy, (1, -1))
        price = np.reshape(price, (1, -1))
    
    load_energy = preprocess_tool.sc_energy.inverse_transform(load_energy) # pu to kWh
    pv_energy = preprocess_tool.sc_energy.inverse_transform(pv_energy) # pu to kWh
    battery_energy = preprocess_tool.sc_energy.inverse_transform(battery_energy) # pu to kWh
    price = preprocess_tool.sc_price.inverse_transform(price) / 100 # pu to cents/kWh to $/kWh
    
    # Compute the transaction cost
    net_energy = load_energy - pv_energy + battery_energy
    cost_transaction = price * net_energy
    
    # Compute the wear and tear cost
    if battery:
        cost_wear = preprocess_tool.sc_price.inverse_transform(battery.wear_cost) * battery_energy
    else:
        cost_wear = 0
            
    return np.sum(cost_transaction + cost_wear)

# Main

In [5]:
# Declare preprocessing tool which contains the trained scalers and 
# functions for feature engineering and formatting for prediction
preprocess_tool = preprocess.preprocess()

# Declare the predictive model for load, price and pv
model_load, model_price, model_pv = predictive_model.retrieve_model()

# Declare the reinforcement learning model for action decision
model_averse, model_seeking = rl_model.retrieve_model()

# Declare battery used in respective cases
# Solution solved using linear programming is continuous
# Solution solved using reinforcement learing is discrete
battery_lp_perfect = ess.battery_continuous()
battery_lp_predict = ess.battery_continuous()
battery_rl_averse = ess.battery_discrete()
battery_rl_seeking = ess.battery_discrete()

In [6]:
# Parameters used in prediction
timesteps_in = 336 # data two week prior to the target
timesteps_out = 24

## Influx

1. The influx class DataFrameClient is used in the project. The DataFrameClient instantiates InfluxDBClient to connect to the backend. The DataFrameClient object holds information necessary to connect to InfluxDB. Requests can be made to InfluxDB directly through the client. The client reads and writes from pandas.

2. https://influxdb-python.readthedocs.io/en/latest/api-documentation.html

In [321]:
class influx():
    # Class variable
    measurement1 = 'sensor'
    measurement2 = 'result'
    
    # Initialize the influx client
    def __init__(self, host='localhost', port=8086, \
                user='ChongAih', password='HCA*1222*dec',\
                db='energy'):
        self.db = db
        self.myclient = influxdb.DataFrameClient(host, port, user, password)
        self.myclient.create_database(db)
        self.myclient.switch_database(db)
    
    # Query from the database and the selected measurement
    # precision in nano second and any input timestamp needs to be converted from s to ns
    def query(self, query):
        results = self.myclient.query(query=query, epoch='ns')
        if results:
            column = next(iter(results)) # measurements/table: dataframe, 0:[], 1:[]
            df = results[column] # Return type is a dataframe
            df.index = df.index.tz_convert(tz='Asia/Singapore') # Convert the timezone of the DateTimeindex
            return df
        else:
            return None
    
    # Write into the database and the selected measurement
    # For bulk data, SeriesHelper will be much effective
    def write(self, df, measurement, tag_columns=None, field_columns=None):
        df.index = df.index.tz_localize(tz='Asia/Singapore') # Set to local timezone
        # df must be a dataframe with DateTimeIndex
        self.myclient.write_points(dataframe=df, measurement=measurement, \
                              tag_columns=tag_columns, field_columns=field_columns)
    
    # For demonstration, the database is dropped each end of demonstration
    def reset(self):
        self.myclient.drop_database(self.db)

In [226]:
""" TO BE REPLACED WITH REAL SENSORS' INPUTS """

# For simulation and demonstration purpose, the existing csv data is fed into the database.
# In actual setting, this section should be replaced with real sensors' inputs
class data_generator():
    def __init__(self, sampling_interval=5):
        self.idx_now = 0
        self.sampling_interval = sampling_interval # seconds
        self.df = pd.read_csv('Final Modified Data_Rev2.csv')
        self.t_current = datetime.datetime.now() - datetime.timedelta(seconds=timesteps_in * sampling_interval)
    
    # Initialize the data in the database
    def get_initial_data(self):
        # The starting timing is set (Not real timing)
        # The database updated each 5 seconds instead of each hour
        times = [self.t_current + datetime.timedelta(seconds=self.sampling_interval * i) \
                for i in range(timesteps_in)]
        times = pd.Series(times, name='time')

        # Concatenate the time and the data from the dataframe
        df_initial = pd.concat([self.df[self.idx_now:self.idx_now+timesteps_in], times], axis=1)
        df_initial.set_index('time', inplace=True, drop=True)
        
        # Update to keep track inserted data
        self.idx_now += timesteps_in
        self.t_current = times.iloc[-1]
        
        return df_initial # Return a dataframe
    
    def get_subsequent_data(self):
        # The database updated each 5 seconds instead of each hour
        times = [self.t_current + datetime.timedelta(seconds=self.sampling_interval)]
        times = pd.Series(times, name='time')
        
        # Reset index as times idx will always start from 0
        df_current = self.df[self.idx_now:self.idx_now+1].reset_index(inplace=False, drop=True)
        
        # Concatenate the time and the data from the dataframe
        df_cont = pd.concat([df_current, times], axis=1)
        df_cont.set_index('time', inplace=True, drop=True)
                
        # Update to keep track inserted data
        self.idx_now += 1
        self.t_current = times.iloc[-1]
        
        return df_cont # Return a dataframe        

In [ ]:
# Initialize battery for different cases
battery_lp_predict = ess.battery_continuous()
battery_rl_averse = ess.battery_discrete()
battery_rl_seeking = ess.battery_discrete()

# Initialize influx DataFrameClient for query and write and generator for simulation
influx_app = influx()
generator = data_generator()

# Initialize the data in the database
influx_app.write(generator.get_initial_data(), influx_app.measurement1)

try:
    while generator.idx_now < len(generator.df):

        influx_app.write(generator.get_subsequent_data(), influx_app.measurement1)

        """Retrive data from influx database"""
        # Retrieve the past two weeks hourly data for LP with prediction and the past 24 hour data for RL
        t_end = generator.t_current
        t_end = datetime.datetime.timestamp(t_end) * 1000000000 # precision is in nanosecond
        t_start = generator.t_current - datetime.timedelta(seconds=generator.sampling_interval * timesteps_in)
        t_start = datetime.datetime.timestamp(t_start) * 1000000000
        query = 'select * from {} where time >{:f} and time <={:f}'.format(influx_app.measurement1, t_start, t_end)
        df = influx_app.query(query)

        # Feature engineering and normalization
        df = preprocess_tool.feature_engineering(df)
        x = preprocess_tool.normalize_pv_load_price(df)

        pv_current, load_current, price_current = np.array([x[-1, 0]]), np.array([x[-1, 1]]), np.array([x[-1, 2]])

        """WITHOUT ESS"""
        cost_woess = compute_benefit(pv_current, load_current, price_current, np.array([0.])) 

        """WITH ESS AND MODEL PREDICTIVE CONTROL (LINEAR PROGRAMMING WITH PREDICTED KNOWLEDGE)"""
        # (336,) + (336, 40) --> (336, 41) --> (1, 336, 41)
        x_pv = np.concatenate((np.expand_dims(x[:,0], axis=1), df.iloc[:, 12:].values), axis=-1)
        x_pv = np.expand_dims(x_pv, axis=0)
        x_load = np.concatenate((np.expand_dims(x[:,1], axis=1), df.iloc[:, 12:].values), axis=-1)
        x_load = np.expand_dims(x_load, axis=0)
        x_price = np.concatenate((np.expand_dims(x[:,2], axis=1), df.iloc[:, 12:].values), axis=-1)
        x_price = np.expand_dims(x_price, axis=0)

        # Predict the future 24 hour load, price and pv
        load_pred = model_load.predict(x_load[:, :, :], False)
        load_pred = np.array(load_pred[0])[0, :, 0]
        price_pred = model_price.predict(x_price[:, :, :], False)
        price_pred = np.array(price_pred[0])[0, :, 0]
        pv_pred = model_pv.predict(x_pv[:, :, :])
        pv_pred = pv_pred[0, :, 0]

        # Update SOC and store SOC, cost & action
        # Use actual pv, load, price for cost computation
        action_predict = optimize_linprog(pv_pred, load_pred, price_pred, battery_lp_predict, timesteps_out)[0]
        SOC_predict = battery_lp_predict.update_SOC(action_predict)
        cost_wess_predict = compute_benefit(pv_current, load_current, price_current, 
                                            np.array([action_predict])*battery_lp_predict.P_rated, 
                                            battery_lp_predict)

        """WITH ESS AND REINFORCEMENT LEARNING (RISK AVERSE AND RISK SEEKING)"""
        price_avg = np.mean(x[-timesteps_out-1:-1, 2])

        # per unit pv, load, price, SOC, average price of past 24 hours - 2D array
        state_averse = np.expand_dims(np.concatenate((pv_current, load_current, price_current, \
                                                      np.array([battery_rl_averse.current_SOC]), \
                                                      np.array([price_avg]))), axis=0)
        state_seeking = np.expand_dims(np.concatenate((pv_current, load_current, price_current, \
                                                      np.array([battery_rl_seeking.current_SOC]), \
                                                      np.array([price_avg]))), axis=0)

        # Discrete solution ranging from [-1, 1] with interval of 0.2
        action_averse = battery_rl_averse.action_set[np.argmax(model_averse.predict(state_averse))]
        action_seeking = battery_rl_seeking.action_set[np.argmax(model_averse.predict(state_seeking))]

        # TO BE REPLACED WITH INFLUX/ SQL STORE DATA
        # Update SOC and store SOC, cost & action
        SOC_averse = battery_rl_averse.update_SOC(action_averse)
        SOC_seeking = battery_rl_seeking.update_SOC(action_seeking)
        cost_wess_averse = compute_benefit(pv_current, load_current, price_current, 
                                           np.array([action_averse])*battery_rl_averse.P_rated, 
                                           battery_rl_averse)
        cost_wess_seeking = compute_benefit(pv_current, load_current, price_current, 
                                            np.array([action_seeking])*battery_rl_seeking.P_rated, 
                                            battery_rl_seeking)

        """Insert data into the database"""
        df_result = pd.DataFrame({'cost ($)': [cost_woess, cost_wess_predict, cost_wess_averse, cost_wess_seeking],
                                  'action': [0, action_predict, action_averse, action_seeking],
                                  'soc': [0, SOC_predict, SOC_averse, SOC_seeking],
                                  'case': ['WOESS', 'ESS_MPC', 'ESS_RLRA', 'ESS_RLRS'],
                                  'time': [generator.t_current] * 4})
        df_result.set_index('time', inplace=True, drop=True)
        # Use tag columns when the timestamp is the same for different cases
        influx_app.write(df=df_result, measurement=influx_app.measurement2, tag_columns=['case'])

        print (datetime.datetime.strftime(generator.t_current, '%Y/%m/%d %H:%M:%S'), \
               SOC_predict, SOC_averse, SOC_seeking)

        time.sleep(2)

except Exception as e:
    print (e)
    
finally:
    influx_app.reset()

2020/07/08 15:17:35 0.5 0.7 0.7
2020/07/08 15:17:40 0.5 0.8999999999999999 0.8999999999999999
2020/07/08 15:17:45 0.5000000000000002 0.94 0.94
2020/07/08 15:17:50 0.7000000000000002 0.94 0.94
2020/07/08 15:17:55 0.9000000000000001 0.94 0.94
2020/07/08 15:18:00 0.7000000000000002 0.94 0.94
2020/07/08 15:18:05 0.5000000000000002 0.94 0.94
2020/07/08 15:18:10 0.5 0.94 0.94
2020/07/08 15:18:15 0.5 0.94 0.94
2020/07/08 15:18:20 0.5 0.94 0.94
2020/07/08 15:18:25 0.5 0.94 0.94
2020/07/08 15:18:30 0.6000000000000002 0.94 0.94
2020/07/08 15:18:35 0.8000000000000002 0.94 0.94
2020/07/08 15:18:40 1.0 0.94 0.94
2020/07/08 15:18:45 1.0 0.94 0.94
2020/07/08 15:18:50 1.0 0.94 0.94
2020/07/08 15:18:55 0.9 0.86 0.86
2020/07/08 15:19:00 0.7000000000000001 0.66 0.66
2020/07/08 15:19:05 0.5000000000000001 0.5800000000000001 0.5800000000000001
2020/07/08 15:19:10 0.5 0.5000000000000001 0.5000000000000001
2020/07/08 15:19:15 0.5 0.5000000000000001 0.5000000000000001
2020/07/08 15:19:20 0.5 0.620000000000000

2020/07/08 15:28:35 0.8000000000000002 0.7000000000000012 0.7000000000000012
2020/07/08 15:28:40 1.0 0.7000000000000012 0.7000000000000012
2020/07/08 15:28:45 1.0 0.7000000000000012 0.7000000000000012
2020/07/08 15:28:50 1.0 0.7000000000000012 0.7000000000000012
2020/07/08 15:28:55 0.8 0.6200000000000012 0.6200000000000012
2020/07/08 15:29:00 0.6000000000000001 0.5400000000000013 0.5400000000000013
2020/07/08 15:29:05 0.5 0.5400000000000013 0.5400000000000013
2020/07/08 15:29:10 0.5 0.5400000000000013 0.5400000000000013
2020/07/08 15:29:15 0.5 0.5400000000000013 0.5400000000000013
2020/07/08 15:29:20 0.5 0.6600000000000013 0.6600000000000013
2020/07/08 15:29:25 0.5 0.8600000000000012 0.8600000000000012
2020/07/08 15:29:30 0.5 0.9400000000000013 0.9400000000000013
2020/07/08 15:29:35 0.5 0.9400000000000013 0.9400000000000013
2020/07/08 15:29:40 0.7 0.9400000000000013 0.9400000000000013
2020/07/08 15:29:45 0.8999999999999999 0.9400000000000013 0.9400000000000013
2020/07/08 15:29:50 1.0 0

2020/07/08 15:39:05 0.7 0.6200000000000033 0.6200000000000033
2020/07/08 15:39:10 0.5 0.5800000000000033 0.5800000000000033
2020/07/08 15:39:15 0.5 0.6600000000000034 0.6600000000000034
2020/07/08 15:39:20 0.5 0.8600000000000033 0.8600000000000033
2020/07/08 15:39:25 0.5 0.9400000000000034 0.9400000000000034
2020/07/08 15:39:30 0.5 0.9400000000000034 0.9400000000000034
2020/07/08 15:39:35 0.5 0.9800000000000034 0.9800000000000034
2020/07/08 15:39:40 0.6000000000000002 0.9800000000000034 0.9800000000000034
2020/07/08 15:39:45 0.8000000000000002 0.9800000000000034 0.9800000000000034
2020/07/08 15:39:50 1.0 0.9800000000000034 0.9800000000000034
2020/07/08 15:39:55 1.0 0.9800000000000034 0.9800000000000034
2020/07/08 15:40:00 0.8 0.9800000000000034 0.9800000000000034
2020/07/08 15:40:05 0.6000000000000001 0.9800000000000034 0.9800000000000034
2020/07/08 15:40:10 0.5 0.9800000000000034 0.9800000000000034
2020/07/08 15:40:15 0.5 0.9800000000000034 0.9800000000000034
2020/07/08 15:40:20 0.5 0

2020/07/08 15:49:30 0.5 0.8600000000000046 0.8600000000000046
2020/07/08 15:49:35 0.5 0.9400000000000047 0.9400000000000047
2020/07/08 15:49:40 0.7 0.9400000000000047 0.9400000000000047
2020/07/08 15:49:45 0.8999999999999999 0.9400000000000047 0.9400000000000047
2020/07/08 15:49:50 1.0 0.9400000000000047 0.9400000000000047
2020/07/08 15:49:55 1.0 0.9400000000000047 0.9400000000000047
2020/07/08 15:50:00 1.0 0.9400000000000047 0.9400000000000047
2020/07/08 15:50:05 0.8 0.7800000000000047 0.7800000000000047
2020/07/08 15:50:10 0.6000000000000001 0.5800000000000047 0.5800000000000047
2020/07/08 15:50:15 0.5 0.5400000000000047 0.5400000000000047
2020/07/08 15:50:20 0.5 0.5400000000000047 0.5400000000000047
2020/07/08 15:50:25 0.7 0.5400000000000047 0.5400000000000047
2020/07/08 15:50:30 0.6000000000000002 0.5400000000000047 0.5400000000000047
2020/07/08 15:50:35 0.8000000000000002 0.5400000000000047 0.5400000000000047
2020/07/08 15:50:40 1.0 0.6600000000000047 0.6600000000000047
2020/07/08

2020/07/08 16:00:00 0.9 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:05 0.7000000000000001 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:10 0.5000000000000001 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:15 0.5 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:20 0.5 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:25 0.5 0.9400000000000059 0.9400000000000059
2020/07/08 16:00:30 0.5 0.860000000000006 0.860000000000006
2020/07/08 16:00:35 0.7 0.940000000000006 0.940000000000006
2020/07/08 16:00:40 0.8999999999999999 0.940000000000006 0.940000000000006
2020/07/08 16:00:45 1.0 0.940000000000006 0.940000000000006
2020/07/08 16:00:50 1.0 0.940000000000006 0.940000000000006
2020/07/08 16:00:55 0.9 0.940000000000006 0.940000000000006
2020/07/08 16:01:00 0.7000000000000001 0.7400000000000061 0.7400000000000061
2020/07/08 16:01:05 0.5000000000000001 0.5400000000000061 0.5400000000000061
2020/07/08 16:01:10 0.5 0.5400000000000061 0.5400000000000061
2020/07

2020/07/08 16:10:30 0.6000000000000002 0.5400000000000071 0.5400000000000071
2020/07/08 16:10:35 0.8000000000000002 0.5400000000000071 0.5400000000000071
2020/07/08 16:10:40 1.0 0.5400000000000071 0.5400000000000071
2020/07/08 16:10:45 1.0 0.5400000000000071 0.5400000000000071
2020/07/08 16:10:50 1.0 0.5400000000000071 0.5400000000000071
2020/07/08 16:10:55 0.8 0.5400000000000071 0.5400000000000071
2020/07/08 16:11:00 0.6000000000000001 0.5400000000000071 0.5400000000000071
2020/07/08 16:11:05 0.5 0.5400000000000071 0.5400000000000071
2020/07/08 16:11:10 0.5 0.5400000000000071 0.5400000000000071
2020/07/08 16:11:15 0.5 0.5400000000000071 0.5400000000000071
2020/07/08 16:11:20 0.5 0.6600000000000071 0.6600000000000071
2020/07/08 16:11:25 0.5 0.8600000000000071 0.8600000000000071
2020/07/08 16:11:30 0.5 0.9400000000000072 0.9400000000000072
2020/07/08 16:11:35 0.5 0.9400000000000072 0.9400000000000072
2020/07/08 16:11:40 0.7 0.9400000000000072 0.9400000000000072
2020/07/08 16:11:45 0.899